In [1]:
import json
import pandas as pd

In [3]:
df=pd.read_json('./dataResume.json')

In [5]:
train_data = []
for index, row in df.iterrows():
    train_data.append({
        "input_text": row["Resume_str"],
        "target_text": json.dumps(row["ner_resume"], ensure_ascii=False)
    })

# حفظها كملف JSON
with open("resume_train.json", "w", encoding="utf-8") as f:
    for item in train_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

model_name = r"D:\LaMini-Flan"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,             
    lora_alpha=32,
    target_modules=["q", "v"], 
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = load_dataset("json", data_files={"train": "resume_train.json"}, split="train")

max_input_length = 1024
max_target_length = 512

def preprocess(example):
    inputs = tokenizer(example["input_text"], max_length=max_input_length, truncation=True)
    targets = tokenizer(example["target_text"], max_length=max_target_length, truncation=True)
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": targets["input_ids"]}

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


Generating train split: 16 examples [00:00, 130.77 examples/s]
Map: 100%|██████████| 16/16 [00:00<00:00, 69.43 examples/s]


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_lora_resume",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=10,
    logging_steps=5,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15048\671714010.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
